# Peer Review Toronto

## Part 1 Intitial Setup (libraires and importing the dataframe)

In [1]:
#Importing Libraries
import json
import requests
import pandas as pd
import geocoder
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize # tranform JSON file into a pandas 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

In [2]:
#Part 2 importing csv(The generated in Capstone 3-2) 
df=pd.read_csv('TLL.csv')

In [130]:

df.tail(5)

,PCode,Borough,Neighborhood,Latitude,Longitude
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
102,M9W,Etobicoke,Northwest,43.706748,-79.594054


In [4]:
# I was handling some problems while tryng to get Toronto Coordinates so i got them from internet
latitude=43.653963
longitude=-79.387207

## Part 2 Generating map

In [5]:
# create map of Toronto using latitude and longitude values
map1= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map1)  
    
map1



### As you can see the map shows the coordinate points of the Borough

## Part 3 Defining Parameters for last part

### Defining 4Square 

In [6]:
## Define Foursquare Credentials and Version
CLIENT_ID = 'BTDVXGUZAWHIDRCOFFDJO1EH3L5HA3Y2BLE4BSWNDYVYLM03' # your Foursquare ID
CLIENT_SECRET = '4LGW4PEZ1HIXT3MNGK4ZDNNX53T4GP5G0CNVCJMRGHZ0BMIT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [7]:
df[df['Neighborhood']=='Rosedale'].index

Int64Index([50], dtype='int64')

In [8]:
NLAT = df.loc[50,'Latitude'] # neighborhood latitude value
NLOT = df.loc[50,'Longitude'] # neighborhood longitude value

NName =df.loc[50 ,'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(NName, 
                                                               NLAT, 
                                                               NLOT))



Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


In [9]:
LIMIT = 180
radius = 1500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?client_id=BTDVXGUZAWHIDRCOFFDJO1EH3L5HA3Y2BLE4BSWNDYVYLM03&client_secret=4LGW4PEZ1HIXT3MNGK4ZDNNX53T4GP5G0CNVCJMRGHZ0BMIT&ll=43.653963,-79.387207&v=20180605&radius=1500&limit=180'

In [10]:
results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '5ce061c04c1f6753b6eeef67'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-5227bb01498e17bf485e6202-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_',
          'suffix': '.png'},
         'id': '4f2a25ac4b909258e854f55f',
         'name': 'Neighborhood',
         'pluralName': 'Neighborhoods',
         'primary': True,
         'shortName': 'Neighborhood'}],
       'id': '5227bb01498e17bf485e6202',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 174,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          'lng': -79.38529600606677}],
        'lat': 43.6532

In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Textile Museum of Canada,Art Museum,43.654396,-79.386500
2,Nathan Phillips Square,Plaza,43.652270,-79.383516
3,Aboveground Art Supplies,Arts & Crafts Store,43.652646,-79.390925
4,Four Seasons Centre for the Performing Arts,Concert Hall,43.650592,-79.385806


In [13]:


print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))



100 venues were returned by Foursquare.


## A brief explanation , after using your credentials you must take in account this 2 parameters ( radius and limit) which esablish the maximum distance for searching venues with the coordinates as a point of start , and limit hte maximum number of venues that will be fecthed from 4square API, in this case increasing the radius will increase the number of venues found 

## Here a function will be generated to get the nearby venues for each neigborhood

In [14]:
def gNV(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhoods', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
TVenues = gNV(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

In [16]:
countf=TVenues.filter(['Neighborhoods', 'Venue'])
countf=countf.groupby('Neighborhoods').count()
countf.head(10)

,Venue
Neighborhoods,
"Adelaide,King,Richmond",100
Agincourt,5
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",3
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",9
"Alderwood,Long Branch",8
"Bathurst Manor,Downsview North,Wilson Heights",18
Bayview Village,4
"Bedford Park,Lawrence Manor East",22
Berczy Park,57


In [17]:
print('There are {} uniques categories.'.format(len(TVenues['Venue Category'].unique())))

There are 278 uniques categories.


In [71]:
# one hot encoding
tor_onehot = pd.get_dummies(TVenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhoods'] =TVenues['Neighborhoods'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

,Neighborhoods,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
tor_onehot.shape

(2239, 279)

In [73]:

tor_grouped = tor_onehot.groupby('Neighborhoods').mean().reset_index()
tor_grouped

,Neighborhoods,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.010000,0.000000,0.000000,0.000000,0.0000,0.010000,0.0,0.000000,0.000000,0.000000
1,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
4,"Alderwood,Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
5,"Bathurst Manor,Downsview North,Wilson Heights",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.055556,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
6,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
7,"Bedford Park,Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
8,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.017544,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000
9,"Birch Cliff,Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000


In [74]:
num_top_venues = 5

for hood in tor_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2                  Bar  0.04
3  American Restaurant  0.04
4      Thai Restaurant  0.04


----Agincourt----
            venue  freq
0  Sandwich Place   0.2
1    Skating Rink   0.2
2  Breakfast Spot   0.2
3      Print Shop   0.2
4          Lounge   0.2


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                        venue  freq
0                  Playground  0.33
1                 Coffee Shop  0.33
2                        Park  0.33
3           Accessories Store  0.00
4  Modern European Restaurant  0.00


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0        Grocery Store  0.22
1          Pizza Place  0.11
2          Coffee Shop  0.11
3           Beer Store  0.11
4  Fried Chicken Joint  0.11


----Alderwood,Long Branch----
            venue  freq
0     Pi



----Downsview West----
                        venue  freq
0               Moving Target  0.25
1                        Bank  0.25
2               Shopping Mall  0.25
3               Grocery Store  0.25
4  Modern European Restaurant  0.00


----Downsview,North Park,Upwood Park----
                        venue  freq
0            Basketball Court  0.25
1                      Bakery  0.25
2  Construction & Landscaping  0.25
3                        Park  0.25
4           Accessories Store  0.00


----East Birchmount Park,Ionview,Kennedy Park----
              venue  freq
0    Discount Store  0.33
1        Hobby Shop  0.17
2  Department Store  0.17
3       Bus Station  0.17
4       Coffee Shop  0.17


----East Toronto----
                 venue  freq
0                 Park   0.4
1          Coffee Shop   0.2
2    Convenience Store   0.2
3  Rental Car Location   0.2
4    Accessories Store   0.0


----Emery,Humberlea----
                        venue  freq
0              Baseball Field   1

               venue  freq
0        Coffee Shop  0.08
1        Pizza Place  0.08
2               Café  0.08
3   Sushi Restaurant  0.05
4  Electronics Store  0.05


----Ryerson,Garden District----
                       venue  freq
0                Coffee Shop  0.10
1             Clothing Store  0.06
2                       Café  0.04
3             Cosmetics Shop  0.04
4  Middle Eastern Restaurant  0.03


----Scarborough Village----
               venue  freq
0         Playground   0.5
1      Jewelry Store   0.5
2  Mobile Phone Shop   0.0
3      Movie Theater   0.0
4              Motel   0.0


----Silver Hills,York Mills----
                        venue  freq
0                   Cafeteria   1.0
1           Accessories Store   0.0
2  Modern European Restaurant   0.0
3                      Museum   0.0
4               Moving Target   0.0


----St. James Town----
            venue  freq
0     Coffee Shop  0.07
1            Café  0.06
2           Hotel  0.05
3      Restaurant  0.05
4  Cosm

In [75]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [128]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhoods']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.tail()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,Willowdale West,Pizza Place,Coffee Shop,Discount Store,Butcher,Pharmacy,Grocery Store,Colombian Restaurant,Department Store,Ethiopian Restaurant,Empanada Restaurant
96,Woburn,Coffee Shop,Korean Restaurant,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
97,"Woodbine Gardens,Parkview Hill",Pizza Place,Fast Food Restaurant,Gastropub,Pharmacy,Gym / Fitness Center,Rock Climbing Spot,Bank,Intersection,Athletics & Sports,Pet Store
98,Woodbine Heights,Skating Rink,Park,Video Store,Beer Store,Asian Restaurant,Curling Ice,Cosmetics Shop,Athletics & Sports,Pharmacy,Concert Hall
99,York Mills West,Park,Convenience Store,Bank,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio


In [126]:


## Cluster Neighborhoods
## Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_grouped_clustering = tor_grouped.drop('Neighborhoods',1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
len(kmeans.labels_)
len(toronto_grouped_clustering)
tor_grouped['Neighborhoods']


0                                Adelaide,King,Richmond
1                                             Agincourt
2     Agincourt North,L'Amoreaux East,Milliken,Steel...
3     Albion Gardens,Beaumond Heights,Humbergate,Jam...
4                                 Alderwood,Long Branch
5         Bathurst Manor,Downsview North,Wilson Heights
6                                       Bayview Village
7                      Bedford Park,Lawrence Manor East
8                                           Berczy Park
9                            Birch Cliff,Cliffside West
10    Bloordale Gardens,Eringate,Markland Wood,Old B...
11           Brockton,Exhibition Place,Parkdale Village
12    Business Reply Mail Processing Centre 969 Eastern
13                           CFB Toronto,Downsview East
14    CN Tower,Bathurst Quay,Island airport,Harbourf...
15                           Cabbagetown,St. James Town
16                                  Caledonia-Fairbanks
17                Canada Post Gateway Processing

In [125]:
tor_merged = df
# add clustering labels


tor_merged['Cluster Labels'] = kmeans.labels_


# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tor_merged.head() # check the l;ast columns!

ValueError: Length of values does not match length of index

In [131]:
tor_merged.tail()

,PCode,Borough,Neighborhood,Latitude,Longitude
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
102,M9W,Etobicoke,Northwest,43.706748,-79.594054


In [106]:
tor_merged.head(8)

,PCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,Fast Food Restaurant,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Spa,Medical Center,Rental Car Location,Mexican Restaurant,Breakfast Spot,Intersection,Electronics Store,Pizza Place,Eastern European Restaurant,Dumpling Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,Coffee Shop,Korean Restaurant,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Hakka Restaurant,Caribbean Restaurant,Bank,Bakery,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Convenience Store,Comic Shop,Event Space
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,Playground,Jewelry Store,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,Discount Store,Department Store,Bus Station,Hobby Shop,Coffee Shop,Donut Shop,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,Bakery,Bus Line,Soccer Field,Fast Food Restaurant,Intersection,Bus Station,Metro Station,Doner Restaurant,Discount Store,Dog Run


In [97]:
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
ys


[array([0, 1, 2, 3, 4]),
 array([ 1,  3,  7, 13, 21]),
 array([ 2,  7, 20, 41, 70]),
 array([  3,  13,  41,  87, 151]),
 array([  4,  21,  70, 151, 264])]

In [101]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

KeyError: 'Cluster Labels'

In [89]:
tor_merged


,PCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,Fast Food Restaurant,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Spa,Medical Center,Rental Car Location,Mexican Restaurant,Breakfast Spot,Intersection,Electronics Store,Pizza Place,Eastern European Restaurant,Dumpling Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,Coffee Shop,Korean Restaurant,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Hakka Restaurant,Caribbean Restaurant,Bank,Bakery,Fried Chicken Joint,Thai Restaurant,Athletics & Sports,Convenience Store,Comic Shop,Event Space
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,Playground,Jewelry Store,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,Discount Store,Department Store,Bus Station,Hobby Shop,Coffee Shop,Donut Shop,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,Bakery,Bus Line,Soccer Field,Fast Food Restaurant,Intersection,Bus Station,Metro Station,Doner Restaurant,Discount Store,Dog Run
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,Motel,American Restaurant,Yoga Studio,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,College Stadium,Café,Skating Rink,General Entertainment,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
